# Code With Me - Simpy - 01 - Generators

Simpy uses Pyhton generators.
> Processes in SimPy are defined by Python generator functions and may, for example, be used to model active components like customers, vehicles or agents. - Simpy documentation 

The goal of this episode is to understand what is a generator and how does simpy use generators.

Feel free to skip the first part it you know already what a generator is, and all this episaode if you do not feed the need to go deeper into details about generators.

### Goal
- understand Python iterators and generator
- understand how Simpy uses generators

### Install the simpy package

In [23]:
!pip install simpy

## the yield keyword

Remember the process we wrote in the basic Simpy example.

In [1]:
def car(env):
    while True:
        print('driver moves to the pump at %d' % env.now)
        # it takes 1 step to move in front of the pump
        yield env.timeout(1)

        print('driver refill the tank at %d' % env.now)
        # it takes 3 steps (says minutes) to fill the tank
        yield env.timeout(3)
        
        print('driver goes away at %d' % env.now)
        # it takes 1 step to go back into the car and leave the gas station
        yield env.timeout(1)


The process uses yield to wait for the next action

*yield* is the keywork Python uses to create generators.

From Python documentation, we can learn that a generator is an handy way to build an iterator.

Wait, what is an iterator ?

## What is an iterator ?

You actually used iterators before. For instance, you may go through a list using an iterator. 

In the following case, *for* will ask an iterator from fruits
and iterate over

In [2]:
fruits = [ "Apple", "Orange", "Banana"]
for word in fruits:
    print(word)

Apple
Orange
Banana


One of the advantages of iterators is that you do not need to know the internals of the collection. It might be a list, a set, a custom thing.

For instance, I may iterate over a set of fruits with the same code.

In [3]:
fruits_set = ( "Apple", "Orange", "Banana" )
for word in fruits_set:
    print(word)

Apple
Orange
Banana


The examples above are about words. However, iterators are often used with numbers. 

For instance, generate a list of numbers.

In [4]:
numbers = [ 1, 2, 3, 4 ]
for number in numbers:
    print(number)

sum_numbers = sum(numbers)
print(f"sum {sum_numbers}")

1
2
3
4
sum 10


The modulr Itertools defines a bunch of iterators for common purposes.

Reference: 
https://docs.python.org/3/library/itertools.html

Let us use the iterator repeat to yield an list of the same number repeating 3 times.

In [5]:
from itertools import repeat
for number in repeat(10, 3) :
    print(number)

10
10
10


Iterators are really useful to generate infinite lists. You could not set un list with all numbers. 

For instance, the iterator count generates number starting from start and incremented by an amount defined by step.

Say we want to generate odd numbers. As the list is infinite the for expression will loop forever. We will use a different way to get the iterator values. We will read the iterator values one by one using next.

In [6]:
from itertools import count
odd_number = count(1,2)
print(next(odd_number))
print(next(odd_number))
print(next(odd_number))

1
3
5


## How does an iterator work ?

Internally, the for mecanism that goes through the list is similar to the code snippet below. 

The for expression creates an iterator and call next to get each value until there is no more value to read. 
When fruit list has been drained it raises an exception StopIteration. As a result for stops iterating over the list.

In [7]:
import sys
words = [ "Apple", "Orange", "Banana"]
words_iterator = iter(words)
try:
    print(next(words_iterator))
    print(next(words_iterator))
    print(next(words_iterator))
    print(next(words_iterator))
except StopIteration as e:
    # fourth next raises this exception bexause the list has only 3 items
    print("Unexpected error:", sys.exc_info()[0])

Apple
Orange
Banana
Unexpected error: <class 'StopIteration'>


## How to write an iterator ?

Say I want to create a Fruit Basket and iterate over. What does it requires ?

Let us write a very simple class and try to use for to iterrate over.

In [8]:
class FruitBasket:
    def __init__(self):
        self.fruits = [ "Apple", "Orange", "Banana"]

basket = FruitBasket()
try:
    for fruit in basket:
        print(fruit)
except:
    print("Error:", sys.exc_info())

Error: (<class 'TypeError'>, TypeError("'FruitBasket' object is not iterable"), <traceback object at 0x10a3989c0>)


This does not work. It complains about FruitBasket not being iterable.

Hmmm, what does it mean to be iterable ?

Let us refactor this a bit in order to be iterable.

First iterable defines a behavior. It tells that you can loop over the object. There are basically two ways of looping over the object and provide the iterable behavior:
- provide a mean to get the element at a given index and implement \_\_getitem\_\_
- or provide an iterator and implement \_\_iter\_\_

As we define iterable partly in terms or iterator, what is an iterator ?

An iterator allow to loop over the elements without the need of an index. Instead, it provides a way to get the next element and implement a method \_\_next\_\_. As a consequence, an iterator must maintain the state of the loop by itself.

Let us implement FruitBasket so that it is iterable. The function getitem will read the element at a given position in the list (an index).

For will be in charge of computing each index values 
up to the point where index is out of bound.

In [9]:
class FruitBasketWithIterable:
    def __init__(self):
        self.fruits = [ "Apple", "Orange", "Banana"]
        
    def __getitem__(self, n):
        if (n > len(self.fruits)):
            # n is out of the list upper bound
            raise IndexError
        # Otherwise get the nth fruit 
        return self.fruits[n]

basket = FruitBasketWithIterable()
for fruit in basket:
    print(fruit)

Apple
Orange
Banana


  By the way, methods named with enclosing \_\_ are special. 
  They provide some well known contract. 

  For instance:
  - \_\_init\_\_ construct an object of the class type
  - \_\_repr\_\_ outputs a representation of the pbject as a string

Let us write the iterable the other way. This one is iterable and rely on the iterator of the internal list.

In [10]:
class FruitBasketWithIterator:
    def __init__(self):
        self.fruits = [ "Apple", "Orange", "Banana"]
        
    def __iter__(self):
        return iter(self.fruits)

basket = FruitBasketWithIterator()
for fruit in basket:
    print(fruit)

Apple
Orange
Banana


A third option is to build a custop iterator. 

What would be a FruitBasket iterator ?

An iterator must provide a \_\_next\_\_ method, keep track of  the state of the loop,  
and is self-iterable (meaning returns self as \_\_iter\_\_

In [11]:
class FruitBasketIterator:
    def __init__(self):
        # init items
        self.fruits = [ "Apple", "Orange", "Banana"]
        # init state and boundary
        self.state = 0
        self.limit = len(self.fruits)

    def __iter__(self):
        # self-iterable
        return self

    def __next__(self):
        if self.state >= self.limit:
            # n is out of the list upper bound
            raise StopIteration
        # get the nth value 
        value = self.fruits[self.state]
        # keep track of the state
        self.state += 1
        return value
    
basket = FruitBasketIterator()
for fruit in basket:
    print(fruit)

Apple
Orange
Banana


This way of writing the iterable is required when dealing with infinite lists. 

For instance, let us implement the odd numbers iterator 

In [12]:
class OddNumbers:
    def __init__(self):
        # init 
        self.state = -1

    def __iter__(self):
        return self

    def __next__(self):
        self.state += 2
        return self.state 

odds_iter = OddNumbers()
print(next(odds_iter))
print(next(odds_iter))
print(next(odds_iter))

1
3
5


## And now the generator

Generators provides a convenient way to write iterators.

> The main feature of generator is evaluating the elements on demand. When you call a normal function with a return statement the function is terminated whenever it encounters a return statement. In a function with a yield statement the state of the function is “saved” from the last call and can be picked up the next time you call a generator function. 

Let us compare the OddNumbers implementation with this one -using a generator. 

In [13]:
def odd_numbers_generator():
    n = 1
    while True:
        yield n
        n += 2
        
odds_gen = odd_numbers_generator()
print(next(odds_gen))
print(next(odds_gen))
print(next(odds_gen))

1
3
5


The generator is written in a more compact way. However it is an iterator. It may be used the same way. 

For instance, iterate over geneated first n numbers, print and sum.

In [14]:
def numbers_first_n(n):
    i = 1
    while i <= n:
        yield i
        i += 1

for number in numbers_first_n(4):
    print(number)

sum_numbers = sum(numbers_first_n(4))
print(f"sum {sum_numbers}")

1
2
3
4
sum 10


You may also iterate over lists

In [15]:
def words_generator(): 
    words = [ "Apple", "Orange", "Banana"]
    for w in words:
        yield w 
        
for w in words_generator():
    print(w)

Apple
Orange
Banana


All example above are plain function. The generator may also be a method in a class.

Please note that in order to be iterable the class must implement __iter__ and __next__. Also do not put the generator into next as it would be recreated each time next is called. The state would be reset and it loops forever. Instead write the generator as a function.

In [16]:
class FruitBasket:
    def __init__(self):
        self.fruits = [ "Apple", "Orange", "Banana"]
        self.iterator = self.generator()

    def __iter__(self):
        return self

    def __next__(self):
        return next(self.iterator)
      
    def generator(self):
        for w in self.fruits:
            yield w 

basket = FruitBasket()
print(next(basket))
for fruit in basket:
    print(fruit)

Apple
Orange
Banana


Please note that the self.generator() returns an iterator not a value.

## How does it relates to Simpy events ?

Let us write a simple Simpy generator that we can use as a process.

In [17]:
import simpy

# a function that mimics a visitor's behavior
def visitor(env):
    print(f"Here I am at {env.now}") 
    yield env.timeout(2)
    print(f"I must leave at {env.now}") 
    
env = simpy.Environment()
# prepare the simulation
# create a visitor
env.process(visitor(env))
# run the simulation step by step for 5 steps 
print(f'Simulation starts at {env.now}')
env.run(until=5)
print(f'Simulation stops at {env.now}')

Simulation starts at 0
Here I am at 0
I must leave at 2
Simulation stops at 5


When the simulation runs, it calls next against the generator. 
The code below mimics what the simulation does.

In [18]:
env = simpy.Environment()
# create a visitor
iterator = visitor(env)
# Do not use e,v.process here. The process is not iterable. 
# The simulation iterates over the generator the process is based on
print("start iteration")
try:
    print("1")
    next(iterator)
    print("2")
    next(iterator)
    print("3")
    next(iterator)
    print("4")
    next(iterator)
except:
    print("Error:", sys.exc_info())

start iteration
1
Here I am at 0
2
I must leave at 0
Error: (<class 'StopIteration'>, StopIteration(), <traceback object at 0x10b5caf40>)


**comment**

The visitor generator is called a first time. It prints the first message. When it reaches yield it blocks until its called again.

When the visitor generator is called a second time, it goes on after the yield and prints the second message. 

The generator is drained and raise a StopIteration exception which tells the environment that the process is done.

The generator let the process decide what is coming next. 
The environment uses the generator as a standard 
interface to get the next element.

Please  note that the time do not pass. All events occurs at 0. Int hte simulation the environment will manage the time passing.

## Multiple yields

What happens if the generator has multiple yields ?

Check what heppens with the car process above.

In [19]:
import simpy

# This is the process for each car
# move to the pup, fill the tank, go away
# the car is passed an id when it is created
def car(env):
    while True:
        print(f'driver moves to the pump at {env.now}')
        # it takes 1 step to move in front of the pump
        yield env.timeout(1)

        print(f'driver refill the tank at {env.now}')
        # it takes 3 steps (says minutes) to fill the tank
        yield env.timeout(3)

        print(f'driver goes away at {env.now}')
        # it takes 1 step to go back into the car and leave the gas station
        yield env.timeout(1)


In [20]:
env = simpy.Environment()
# create a visitor
iterator = car(env)
print("start iteration")
try:
    print("1")
    next(iterator)
    print("2")
    next(iterator)
    print("3")
    next(iterator)
    print("4")
    next(iterator)
except:
    print("Error:", sys.exc_info())

start iteration
1
driver moves to the pump at 0
2
driver refill the tank at 0
3
driver goes away at 0
4
driver moves to the pump at 0


**comment**

When the generaotr has multiple yield, 
whether it declared multiple action or run yield inside a loop, 
the behavior is similar. 
Each time a yield is reached the process will halted until next is called.

## Generator chains

In simpy event may wait until some other event completes. 

For instance, visitor might have subprocesses

In [21]:
import simpy

# a function that mimics the action of taking a picture
def take_photo(env):
    yield env.timeout(2)
    print(f"took photo at {env.now}") 

# a function that mimics a visitor's behavior
def visitor(env):
    print(f"Here I am at {env.now}") 
    yield env.timeout(2)
    yield env.process(take_photo(env))
    print(f"I must leave at {env.now}") 
    
env = simpy.Environment()
# prepare the simulation
# create a visitor
env.process(visitor(env))
# run the simulation step by step for 5 steps 
print(f'Simulation starts at {env.now}')
env.run(until=5)
print(f'Simulation stops at {env.now}')

Simulation starts at 0
Here I am at 0
took photo at 4
I must leave at 4
Simulation stops at 5


What happens when a generator is chained with another generator ?

Let us experiment with the fruit basket.

In [22]:
# this generator produces indexes
def numbers_generator():
    i = 0
    while i < 3:
        yield i
        i += 1
    # raise StopIteration when called while i is > 3

# this generator reads fruit at the index given by the first generator
def fruits_generator():
    fruits = [ "Apple", "Orange", "Banana"]
    # setup the index generator
    gen = numbers_generator()
    try:
        # get next index and read fruit until the StopIteration is raised
        while True:
            yield fruits[next(gen)]
    except StopIteration:
        # it is okay. just stop reading
        pass
            
for fruit in fruits_generator():
    print(fruit)

Apple
Orange
Banana
